# CH 9 NEURAL NETWORKS

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
folder = "/gdrive/My Drive/Python Practice/Datasets"

# HANDS-ON ANALYSIS


---
# Use **adult_ch6_training**, **adult_ch6_test** Dataset below.

---

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import statsmodels.tools.tools as stattools

#from tensorflow.keras.models import Sequential         # too much better performance
#from tensorflow.keras.layers import Dense              # too much better performance
#import tensorflow as tf
#np.random.seed(3)
#tf.random.set_seed(3)

adult_train = pd.read_csv(folder + '/adult_ch6_training', delimiter=',')
adult_test = pd.read_csv(folder + '/adult_ch6_test', delimiter=',')

#17
Prepare the data set for neural network modeling by doing the following:<br>
a.  Create a binary variable that equals one if Cap_Gains_Losses is greater than zero, and zero otherwise. Call it CapGainsLossesPositive.<br>
b.  Convert the Marital.status, Income, and CapGainsLossesPositive to factors.

In [ ]:
adult_train["CapGainsLossesPositive"] = list(map(int, adult_train["Cap_Gains_Losses"]>0))
adult_train.head()

In [ ]:
# change CapGainsLosses to binary
adult_train["CapGainsLossesPositive"] = list(map(int, adult_train["Cap_Gains_Losses"]>0))

# change Income to numeric
adult_train["Income_factor"] = list(map(int, adult_train["Income"] == '>50K'))

# convert 'Marital status' to factor
mar_np = np.array(adult_train['Marital status'])
(mar_cat, mar_cat_dict) = stattools.categorical(mar_np, drop=True, dictnames=True)
mar_cat_pd = pd.DataFrame(mar_cat)

# create list of x names, y names
mar_names = list(set(adult_train['Marital status']))
X_names = ['Caps_Gains_Losses']
for i in mar_names:
    X_names.append(i)
y_names = ["<=50K", ">50K"]

# create x_train, y_train
X_train = pd.concat((adult_train['CapGainsLossesPositive'], mar_cat_pd), axis=1)
Y_train = adult_train['Income_factor']
# Y_train = pd.factorize(adult_train['Income'])[0]
X_train.head()

In [ ]:
# change CapGainsLosses to binary
adult_test["CapGainsLossesPositive"] = list(map(int, adult_test["Cap_Gains_Losses"]>0))

# change Income to numeric
adult_test["Income_factor"] = list(map(int, adult_test["Income"] == '>50K'))

# convert 'Marital status' to factor
mar_np = np.array(adult_test['Marital status'])
(mar_cat, mar_cat_dict) = stattools.categorical(mar_np, drop=True, dictnames=True)
mar_cat_pd = pd.DataFrame(mar_cat)

# create list of x names, y names
mar_names = list(set(adult_test['Marital status']))
X_names = ['Caps_Gains_Losses']
for i in mar_names:
    X_names.append(i)
y_names = ["<=50K", ">50K"]

# create x_train, y_train
X_test = pd.concat((adult_test['CapGainsLossesPositive'], mar_cat_pd), axis=1)
Y_test = adult_test['Income_factor']
# Y_test = pd.factorize(adult_test['Income'])[0]

#18
Using the training data set, create a neural network model to predict a customer’s Income using Marital.status and CapGainsLossesPositive. <br>
Call this NNM1 (For neural network Model 1). <br>
Obtain the predicted responses.

In [ ]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)

NNM1 = Sequential()
NNM1.add(Dense(2, input_dim = 6, activation='relu'))
NNM1.add(Dense(1, activation='sigmoid'))
# compile the keras model 
NNM1.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
#NNM1.compile(loss='mse', optimizer='adam',metrics=['accuracy'])
NNM1.fit(X_train,Y_train, epochs=20, batch_size=5)

scores = NNM1.evaluate(X_train, Y_train)
print('\n%s: %.4f%%' % (NNM1.metrics_names[1], scores[1]*100))

In [ ]:
print('\n Accuracy: %.4f' % (NNM1.evaluate(X_train, Y_train)[1]))
print('\n Test Accuracy: %.4f' % (NNM1.evaluate(X_test, Y_test)[1]))

#19
Plot the NNM1 neural network.

In [ ]:
!pip install ann_visualizer

In [ ]:
from ann_visualizer.visualize import ann_viz
ann_viz(NNM1, title='NNM1', view=True)

#20
Evaluate NNM1 using the test data set. <br>
Construct a contingency table to compare the actual and predicted values of Income.


In [ ]:
test_pred = NNM1.predict(X_test).flatten()
adult_pred = np.array([prob > 0.5 and 1 or 0 for prob in test_pred])

crosstab_adult = pd.crosstab(Y_test, adult_pred, rownames=['Actual'], colnames=['Prediction'])
crosstab_adult

#21
Which baseline model do we compare NNM1 against? <br>
Did NNM1 outperform the base- line according to accuracy?

In [ ]:
base_acc = crosstab_adult.sum(axis=1)[0] / sum(crosstab_adult.sum())
NN_acc = (crosstab_adult.iloc[0,0] + crosstab_adult.iloc[1,1]) / sum(crosstab_adult.sum())
print("Baseline model Accuracy: %.4f%%" %base_acc)
print("Neural Network Accuracy: %.4f%%" %NN_acc)

# NN model outperform the baseline model 

In [ ]:
############################################## 여기서 부터 다시 정리 #######################################################

#22
Gather the results (contingency tables) from your earlier modeling of the adult_ch6_ training and adult_ch6_test data sets in the Chapter 6 and Chapter 8 exercises. <br>
From Chapter  6, call the CART model CARTM1 and call the C5.0 model C5M1. <br>
From Chapter 8, call the Naïve Bayes model NBM1.


* CART (CARTM1)

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from graphviz import Source

#folder = "/gdrive/My Drive/DSPR/DataSets"

adult_train = pd.read_csv(folder + "/adult_ch6_training", delimiter=',')
adult_test = pd.read_csv(folder + "/adult_ch6_test", delimiter=',')

In [ ]:
mar_np = np.array(adult_train['Marital status'])
(mar_cat, mar_cat_dict) = stattools.categorical(mar_np, drop=True, dictnames=True)

mar_cat_pd = pd.DataFrame(mar_cat)
X_adult_tree = pd.concat((adult_train['Cap_Gains_Losses'], mar_cat_pd), axis=1)

y_adult_tree = adult_train[['Income']]

X_adult_names_tree = ["Cap_Gains_Losses", "Divorced", "Married","Never-married","Separated", "Widowed"]
y_adult_names_tree = ["<=50K", ">50K"]

CARTM1 = DecisionTreeClassifier(criterion = "gini", max_leaf_nodes = 5).fit(X_adult_tree, y_adult_tree)
export_graphviz(CARTM1, out_file = folder+"/CARTM1.dot", feature_names = X_adult_names_tree, class_names = adult_train['Income'].unique())
### CART model using Training set 
Source.from_file(folder+"/CARTM1.dot")

In [ ]:
### Predict Test set

mar_np_test = np.array(adult_test['Marital status'])
(mar_cat_test, mar_cat_dict_test) = stattools.categorical(mar_np_test, drop=True, dictnames=True)

In [ ]:
mar_cat_test_pd = pd.DataFrame(mar_cat_test)
X_adult_tree_test =  pd.concat((adult_test['Cap_Gains_Losses'], mar_cat_test_pd), axis=1)

y_adult_tree_test = adult_test[['Income']]

X_adult_names_tree = ["Cap_Gains_Losses", "Divorced", "Married","Never-married","Separated", "Widowed"]
y_adult_names_tree = ["<=50K", ">50K"]

adult_CART_pred = CARTM1.predict(X_adult_tree_test)

adult_CART_crosstab = pd.crosstab(adult_test['Income'], adult_CART_pred)
adult_CART_crosstab

In [ ]:
def eval(tab, level):
    GT = sum(tab.sum())
    TAP = tab.sum(1)[1]
    TAN = tab.sum(1)[0]
    TPP = tab.sum(0)[1]
    TP = tab[level[1]][1]
    TN = tab[level[0]][0]

    acc = (TN + TP) / GT
    err = 1 - acc
    sen = TP / TAP
    spe = TN / TAN
    pre = TP / TPP
    
    return [round(acc, 4), round(err,4), round(sen,4), round(spe,4), round(pre,4)]

def Fscore(pre, spe, df):
    val = ((1+df**2) * pre * spe) / ((df**2 * pre) + spe)
    return round(val, 4)

In [ ]:
acc_cart, err_cart, sen_cart, spe_cart, pre_cart = eval(adult_CART_crosstab, y_adult_names_tree)
acc_cart, err_cart, sen_cart, spe_cart, pre_cart

In [ ]:
Fscore(pre_cart, spe_cart, 1), Fscore(pre_cart, spe_cart, 2), Fscore(pre_cart, spe_cart, 0.5)

In [ ]:
measure_cart_01 = pd.DataFrame(eval(adult_CART_crosstab, y_adult_names_tree))
measure_cart_02 = Fscore(pre_cart, spe_cart, 1), Fscore(pre_cart, spe_cart, 2), Fscore(pre_cart, spe_cart, 0.5)
measure_cart_02 = pd.DataFrame(measure_cart_02)
measure_cart_02

eval_cart = pd.concat((measure_cart_01, measure_cart_02), axis=0)
eval_cart.columns = ['CART']
eval_cart.index = ['Accuracy', 'Error rate', 'Sensitivity', 'Specificity', 'Precision', 'F1', 'F2', 'F0.5']
eval_cart

* C5.0 (C5M1)

In [ ]:
C5M1 = DecisionTreeClassifier(criterion="entropy",max_leaf_nodes=5).fit(X_adult_tree, y_adult_tree)
export_graphviz(C5M1, out_file = folder+"/C5M1.dot", feature_names = X_adult_names_tree, class_names = adult_train['Income'].unique())
### C5.0 model using Training set 
Source.from_file(folder+"/C5M1.dot")

In [ ]:
### Predict Test set

adult_C5M1_pred = C5M1.predict(X_adult_tree_test)

adult_C5M1_crosstab = pd.crosstab(adult_test['Income'], adult_C5M1_pred)
adult_C5M1_crosstab

In [ ]:
acc_c5m1, err_c5m1, sen_c5m1, spe_c5m1, pre_c5m1 = eval(adult_C5M1_crosstab, y_adult_names_tree)
acc_c5m1, err_c5m1, sen_c5m1, spe_c5m1, pre_c5m1

In [ ]:
Fscore(pre_c5m1, spe_c5m1, 1), Fscore(pre_c5m1, spe_c5m1, 2), Fscore(pre_c5m1, spe_c5m1, 0.5)

In [ ]:
measure_cm51_01 = pd.DataFrame(eval(adult_C5M1_crosstab, y_adult_names_tree))
measure_cm51_02 = Fscore(pre_c5m1, spe_c5m1, 1), Fscore(pre_c5m1, spe_c5m1, 2), Fscore(pre_c5m1, spe_c5m1, 0.5)
measure_cm51_02 = pd.DataFrame(measure_cm51_02)
measure_cm51_02

eval_cm51 = pd.concat((measure_cm51_01, measure_cm51_02), axis=0)
eval_cm51.columns = ['C5.0']
eval_cm51.index = ['Accuracy', 'Error rate', 'Sensitivity', 'Specificity', 'Precision', 'F1', 'F2', 'F0.5']
eval_cm51

* Naive Bayes (NBM1)

In [ ]:
adult_train.head()

In [ ]:
from sklearn.naive_bayes import MultinomialNB
import statsmodels.tools.tools as stattools

def X_ind_ftn(data, col):
    X_ind = np.array(data[col])
    X_ind = pd.DataFrame(stattools.categorical(X_ind,drop=True, dictnames = False))
    return X_ind

X_Mar_ind = X_ind_ftn(adult_train, "Marital status")
    
X_naive = pd.concat((X_Mar_ind, adult_train['Cap_Gains_Losses']), axis=1)
y_naive = adult_train["Income"]

nb_adult = MultinomialNB().fit(X_naive,y_naive)

In [ ]:
# Test NB algorithm by Test set
X_Mar_ind_test = X_ind_ftn(adult_test, 'Marital status')

X_naive_test = pd.concat((X_Mar_ind_test, adult_test['Cap_Gains_Losses']), axis=1)
y_naive_test = adult_test['Income']

adult_naive_pred = nb_adult.predict(X_naive_test)

In [ ]:
adult_naive_crosstab = pd.crosstab(y_naive_test, adult_naive_pred, rownames=['Actual'], colnames=['Predicted'])
adult_naive_crosstab

In [ ]:
acc_naive, err_naive, sen_naive, spe_naive, pre_naive = eval(adult_naive_crosstab, y_adult_names_tree)
acc_naive, err_naive, sen_naive, spe_naive, pre_naive

In [ ]:
Fscore(pre_naive, spe_naive, 1), Fscore(pre_naive, spe_naive, 2), Fscore(pre_naive, spe_naive, 0.5)

In [ ]:
measure_naive_01 = pd.DataFrame(eval(adult_naive_crosstab, y_adult_names_tree))
measure_naive_02 = Fscore(pre_naive, spe_naive, 1), Fscore(pre_naive, spe_naive, 2), Fscore(pre_naive, spe_naive, 0.5)
measure_naive_02 = pd.DataFrame(measure_naive_02)
measure_naive_02

eval_naive = pd.concat((measure_naive_01, measure_naive_02), axis=0)
eval_naive.columns = ['Naive Bayes']
eval_naive.index = ['Accuracy', 'Error rate', 'Sensitivity', 'Specificity', 'Precision', 'F1', 'F2', 'F0.5']
eval_naive

#23
Compare the NNM1 results with the three models from the previous exercise, according to the following criteria.<br>
Discuss in detail which model performed best and worst according to each criterion.<br>
a.  Accuracy<br>
b.  Sensitivity<br>
c.  Specificity

In [ ]:
pd.concat((eval_cart, eval_cm51, eval_naive), axis=1)
# should have added Neural Network model....

---
# Use **bank_marketing_training**, **bank_marketing_test** Dataset below.

---

#24
Prepare the data set for neural network modeling, including standardizing the variables.


In [ ]:
bank_train = pd.read_csv(folder + "/bank_marketing_training", delimiter=',')
bank_test = pd.read_csv(folder + "/bank_marketing_test", delimiter=',')

In [ ]:
import statsmodels.tools.tools as stattools

# change 'Response' to numeric(binary)
bank_train["response_factor"] = list(map(int, bank_train["response"] == 'yes'))

# convert 'job', 'marital', 'education' to factor
# job
job_np = np.array(bank_train['job'])
(job_cat, job_cat_dict) = stattools.categorical(job_np, drop=True, dictnames=True)
# marital
mari_np = np.array(bank_train['marital'])
(mari_cat, mari_cat_dict) = stattools.categorical(mari_np, drop=True, dictnames=True)
# education
edu_np = np.array(bank_train['education'])
(edu_cat, edu_cat_dict) = stattools.categorical(edu_np, drop=True, dictnames=True)

In [ ]:
bank_train['job'].unique()

In [ ]:
job_cat_pd = pd.DataFrame(job_cat)
mari_cat_pd = pd.DataFrame(mari_cat)
edu_cat_pd = pd.DataFrame(edu_cat)

X_bank = pd.concat((bank_train['age'], job_cat_pd, mari_cat_pd, edu_cat_pd), axis=1)

#X_bank_names = [ ... ]
#Y_bank_names = ['yes', 'no']

In [ ]:
X_bank.head()

In [ ]:
#X_bank.head()
y_bank = bank_train['response_factor']

In [ ]:
model2 = Sequential()
model2.add(Dense(30, input_dim = 25, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model2.fit(X_bank, y_bank, epochs=5, batch_size=1)

In [ ]:
print('\n Accuracy: %.4f' % (model2.evaluate(X_bank, y_bank)[1]))

In [ ]:
bank_test["response_factor"] = list(map(int, bank_test["response"] == 'yes'))

job_np_test = np.array(bank_test['job'])
(job_cat_test, job_cat_dict_test) = stattools.categorical(job_np_test, drop=True, dictnames=True)

mari_np_test = np.array(bank_test['marital'])
(mari_cat_test, mari_cat_dict_test) = stattools.categorical(mari_np_test, drop=True, dictnames=True)

edu_np_test = np.array(bank_test['education'])
(edu_cat_test, edu_cat_dict_test) = stattools.categorical(edu_np_test, drop=True, dictnames=True)

In [ ]:
job_cat_test_pd = pd.DataFrame(job_cat_test)
mari_cat_test_pd = pd.DataFrame(mari_cat_test)
edu_cat_test_pd = pd.DataFrame(edu_cat_test)

X_bank_test = pd.concat((bank_test['age'], job_cat_test_pd, mari_cat_test_pd, edu_cat_test_pd), axis=1)

#X_bank_names = [ ... ]
#Y_bank_names = ['yes', 'no']

In [ ]:
#X_bank_test.head()
y_bank_test = bank_test['response_factor']

In [ ]:
print('\n Test Accuracy: %.4f' % (model2.evaluate(X_bank_test, y_bank_test)[1]))

In [ ]:
y_bank_prediction = model2.predict(X_bank_test).flatten()
bank_pred = pd.concat((pd.DataFrame(y_bank_prediction), y_bank_test), axis=1) 

bank_tab = pd.crosstab(y_bank_test, y_bank_prediction, rownames=['Actual'], colnames=['Prediction'])
bank_tab

In [ ]:
# ????????????????????????? all 1 on prediction? 

#25
Using the training data set, create a neural network model to predict a customer’s Response using whichever predictors you think appropriate. <br>
Obtain the predicted responses.


In [ ]:
y_bank_prediction # all 0 ????

#26
Plot the neural network.

In [ ]:
# ????????????

#27
Evaluate the neural network model using the test data set. <br>
Construct a contingency table to compare the actual and predicted values of Response.


In [ ]:
bank_tab

#28
Which baseline model do we compare your neural network model against? <br>
Did it out- perform the baseline according to accuracy?


In [ ]:
# ???

#29
Using the same predictors you used for your neural network model, build models to pre- dict Response using the following algorithms:<br>
a. CART<br>
b.  C5.0<br>
c.  Naïve Bayes

In [ ]:
bank_train.head() 

In [ ]:
bank_train['education'].unique()

In [ ]:
def X_ind_ftn(data, col):
    X_ind = np.array(data[col])
    X_ind = pd.DataFrame(stattools.categorical(X_ind,drop=True, dictnames = False))
    return X_ind

X_job_ind = X_ind_ftn(bank_train, "job")
X_marital_ind = X_ind_ftn(bank_train, "marital")
X_edu_ind = X_ind_ftn(bank_train, "education")

X_bank_tree = pd.concat((bank_train[['age']],X_job_ind, X_marital_ind, X_edu_ind), axis=1)
X_bank_tree

y_bank_tree = bank_train[['response']]

X_bank_names_tree = ["age", "housemaid", "services","blue-collar","management", "unemployed", "retired", "technician", "admin.","unknown","entrepreneur","student","self-employed",
                     "married","single","divorced","unknown",
                     "basic.4y","high.school","unknown","basic.6y","basic.9y","university.degree","prefessional.course","illiterate"]
y_bank_names_tree = ["yes", "no"]

In [ ]:
CART_bank = DecisionTreeClassifier(criterion = "gini", max_leaf_nodes = 5).fit(X_bank_tree, y_bank_tree)
export_graphviz(CART_bank, out_file = folder+"/CART_bank.dot", feature_names = X_bank_names_tree, class_names = bank_train['response'].unique())
### CART model using Training set 
Source.from_file(folder+"/CART_bank.dot")

In [ ]:
X_job_ind_test = X_ind_ftn(bank_test, "job")
X_marital_ind_test = X_ind_ftn(bank_test, "marital")
X_edu_ind_test = X_ind_ftn(bank_test, "education")

X_bank_tree_test =  pd.concat((bank_test['age'], X_job_ind_test, X_marital_ind_test, X_edu_ind_test), axis=1)

y_bank_tree_test = bank_test[['response']]

bank_CART_pred = CART_bank.predict(X_bank_tree_test)

bank_CART_crosstab = pd.crosstab(bank_test['response'], bank_CART_pred, rownames=["Actual"], colnames=["Predicted"])
bank_CART_crosstab  # no yes??

In [ ]:
acc_bank_cart, err_bank_cart, sen_bank_cart, spe_bank_cart, pre_bank_cart = eval(bank_CART_crosstab, y_bank_names_tree)
acc_bank_cart, err_bank_cart, sen_bank_cart, spe_bank_cart, pre_bank_cart

Fscore(pre_bank_cart, spe_bank_cart, 1), Fscore(pre_bank_cart, spe_bank_cart, 2), Fscore(pre_bank_cart, spe_bank_cart, 0.5)

In [ ]:
bank_cart_01 = pd.DataFrame(eval(bank_CART_crosstab, y_bank_names_tree))
bank_cart_02 = Fscore(pre_bank_cart, spe_bank_cart, 1), Fscore(pre_bank_cart, spe_bank_cart, 2), Fscore(pre_bank_cart, spe_bank_cart, 0.5)
bank_cart_02 = pd.DataFrame(bank_cart_02)
bank_cart_02

In [ ]:
eval_bank_cart = pd.concat((bank_cart_01, bank_cart_02), axis=0)
eval_bank_cart.columns = ['CART']
eval_bank_cart.index = ['Accuracy', 'Error rate', 'Sensitivity', 'Specificity', 'Precision', 'F1', 'F2', 'F0.5']
eval_bank_cart

* C5.0

In [ ]:
C5M1_bank = DecisionTreeClassifier(criterion="entropy",max_leaf_nodes=5).fit(X_bank_tree, y_bank_tree)
export_graphviz(C5M1_bank, out_file = folder+"/C5M1_bank.dot", feature_names = X_bank_names_tree, class_names = bank_train['response'].unique())
### C5.0 model using Training set 
Source.from_file(folder+"/C5M1_bank.dot")

In [ ]:
### Predict Test set

bank_C5M1_pred = C5M1_bank.predict(X_bank_tree_test)

bank_C5M1_crosstab = pd.crosstab(bank_test['response'], bank_C5M1_pred)
bank_C5M1_crosstab # no yes??

In [ ]:
acc_bank_c5m1, err_bank_c5m1, sen_bank_c5m1, spe_bank_c5m1, pre_bank_c5m1 = eval(bank_C5M1_crosstab, y_bank_names_tree)
acc_bank_c5m1, err_bank_c5m1, sen_bank_c5m1, spe_bank_c5m1, pre_bank_c5m1

In [ ]:
Fscore(pre_bank_c5m1, spe_bank_c5m1, 1), Fscore(pre_bank_c5m1, spe_bank_c5m1, 2), Fscore(pre_bank_c5m1, spe_bank_c5m1, 0.5)

In [ ]:
bank_cm51_01 = pd.DataFrame(eval(bank_C5M1_crosstab, y_bank_names_tree))
bank_cm51_02 = Fscore(pre_bank_c5m1, spe_bank_c5m1, 1), Fscore(pre_bank_c5m1, spe_bank_c5m1, 2), Fscore(pre_bank_c5m1, spe_bank_c5m1, 0.5)
bank_cm51_02 = pd.DataFrame(bank_cm51_02)
bank_cm51_02

eval_bank_cm51 = pd.concat((bank_cm51_01, bank_cm51_02), axis=0)
eval_bank_cm51.columns = ['C5.0']
eval_bank_cm51.index = ['Accuracy', 'Error rate', 'Sensitivity', 'Specificity', 'Precision', 'F1', 'F2', 'F0.5']
eval_bank_cm51

* Naive Bayes

In [ ]:
####

#30
Compare the results of your neural network model with the three models from the previous exercise, according to the following criteria. <br>
Discuss in detail which model performed best and worst according to each criterion.<br>
a. Accuracy<br>
b.  Sensitivity<br>
c.  Specificity

In [ ]:
####